In [258]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [259]:
readdata = pd.read_csv("../dataset/heart.csv")

mapping = {value: index for index, value in enumerate(readdata['ChestPain'].unique())}
readdata['ChestPain'] = readdata['ChestPain'].map(mapping)

mapping = {value: index for index, value in enumerate(readdata['Thal'].unique())}
readdata['Thal'] = readdata['Thal'].map(mapping)

readdata.loc[readdata['AHD']=='No','AHD'] = 0
readdata.loc[readdata['AHD']=='Yes','AHD'] = 1

readdata.dropna(inplace=True)

data = readdata.to_numpy()
data = data[:,1:].astype('float64')

k = 10

In [260]:
def fda(tmp,labels):

    class0 = tmp[labels==0]
    class1 = tmp[labels==1]

    cent = np.zeros((2,tmp.shape[1]))
    cent[0] = np.mean(class0,axis=0)
    cent[1] = np.mean(class1,axis=0)

    sw = np.zeros((tmp.shape[1],tmp.shape[1]))
    for j in range(2):
        s_j = np.zeros((tmp.shape[1],tmp.shape[1]))
        for row in tmp[labels==j]:
            row = row.reshape(-1,1)
            mean = cent[j].reshape(-1,1)
            t = (row-mean).dot((row-mean).T).astype('float64')
            s_j += t
        sw += s_j

    mean0 = cent[0].reshape(-1,1)
    mean1 = cent[1].reshape(-1,1)

    sb = (mean0 - mean1).dot((mean0 - mean1).T)
    
    evals,evecs = np.linalg.eig((np.linalg.inv(sw))@sb)
    sel = np.argsort(evals)[::-1]
    evecs = evecs[:,sel]

    y = tmp.dot(evecs)

    return np.column_stack((y,labels))

In [261]:
def pcafda(inp,c):
    labels = inp[:,-1]
    tmp = inp[:,:-1]

    tmp = (tmp - np.mean(tmp,axis=0))/np.std(tmp,axis=0)
    cov = np.cov(tmp.T)

    evals,evecs = np.linalg.eig(cov)
    sel = np.argsort(evals)[::-1]
    evecs = evecs[:, sel]
    evecs = evecs[:,:c]
    tmp = tmp.dot(evecs)

    return fda(tmp,labels)

In [262]:
def logistic_regression(inp):
    def sigmoid(x):
        return 1/(1 + np.exp(-x))
    
    traindata = inp[:int(0.8*len(inp))]
    testdata = inp[int(0.8*len(inp)):]
    Y = traindata[:,-1]
    X = traindata[:,:-1]
    Y_test = testdata[:,-1]
    X_test = testdata[:,:-1]
    n,m = X.shape
    theta = np.zeros((m,1))
    max_iter = 1000
    alpha = 0.02

    for i in range(max_iter):
        p = np.zeros((n,))
        for i in range(n):
            print(theta.shape,X[i].shape)
            p[i] = theta.dot(X[i].reshape(-1,1))
        newtheta = theta - alpha * ((X.T).dot(p-Y))/n
        if(np.abs(newtheta-theta)<=0.001):
            break
        theta = newtheta
    
    probs = sigmoid(X_test.dot(theta))
    pred = (probs>=0.5).astype('int')
    return 100*accuracy_score(Y_test,pred)

In [263]:
fda_only = fda(data[:,:-1],data[:,-1])
pca_fda = pcafda(data,k)

In [264]:
logistic_regression(fda_only)

(13, 1) (13,)


ValueError: shapes (13,1) and (13,1) not aligned: 1 (dim 1) != 13 (dim 0)